In [1]:
import tensorflow as tf
import tensorflow_federated as tff

In [2]:
train_client_spec = tff.simulation.baselines.ClientSpec(
    num_epochs=1,
    batch_size=32
)

task = tff.simulation.baselines.emnist.create_character_recognition_task(
    train_client_spec,
    model_id='cnn',
    only_digits=False
)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [4]:
isinstance(task.model_fn(), tff.learning.Model)


True

In [5]:
task.datasets.summary()

Split |Dataset Type |Number of Clients |
Train |Federated    |3400              |
Test  |Federated    |3400              |
________________________________________
Train Preprocess Function: True
Eval Preprocess Function: True


In [7]:
fed_avg = tff.learning.algorithms.build_weighted_fed_avg(
    task.model_fn,
    client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=0.1),
)

In [8]:
def client_sel(round_num):
    return task.datasets.sample_train_clients(num_clients=5)

In [9]:
tff.simulation.run_training_process(
    fed_avg, client_sel, total_rounds=500
)

2023-02-28 01:26:31.571679: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


KeyboardInterrupt: 

In [ ]:
test_data = task.datasets.get_centralized_test_data()

In [ ]:
eval_process = tff.learning.build_federated_evaluation(task.model_fn)


In [ ]:
def validation_fn(state, round_num):
    return eval_process(state.model, [test_data])


In [ ]:
tff. simulation. run_simulation (
process, client_selection_fn, total_rounds 500, validation_fn=validation_fn)